In [ ]:
import pathlib as pl
import numpy as np
import matplotlib.pyplot as plt
import flopy
import flopy.mf6 as fp6

# Base Model Location

In [ ]:
name = "ex1"
ws = pl.Path("working/single")

In [ ]:
ex_pth = "../../.pixi/env/bin/mf6"

# Problem Dimensions

## Temporal Discretization

In [ ]:
nper = 3
tdis_data = [
    (300000., 1, 1.0),
    (36500., 10, 1.5),
    (300000, 1, 1.0),
]

## Spatial Discretization

In [ ]:
nlay, nrow, ncol = 3, 21, 20

In [ ]:
xlen, ylen = 10000., 10500.

In [ ]:
delc = ylen / nrow
delr = xlen / ncol
delr, delc

In [ ]:
top = 400.
botm = [220., 200., 0]

## Hydraulic Properties

In [ ]:
K11 = [50., 0.01, 200.]
K33 = [10., 0.01, 20.]

## Storage properties

In [ ]:
Ss, Sy = 0.0001, 0.1

## Boundary Condition data

In [ ]:
H_east = 320.

In [ ]:
recharge = 0.005

# Build the model

## MODFLOW Simulation files

In [ ]:
sim = fp6.MFSimulation(sim_name=name, sim_ws=ws, exe_name=ex_pth)

In [ ]:
tdis = fp6.ModflowTdis(sim, nper=nper, perioddata=tdis_data, time_units="days")

In [ ]:
ims = fp6.ModflowIms(sim, complexity="simple", print_option="all")

## Groundwater flow model

In [ ]:
gwf = fp6.ModflowGwf(sim, modelname=name, save_flows=True)

In [ ]:
dis = fp6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, top=top, botm=botm, length_units="meters")

In [ ]:
gwf.modelgrid

In [ ]:
npf = fp6.ModflowGwfnpf(gwf, icelltype=1, save_specific_discharge=True, k=K11, k33=K33)

In [ ]:
sto = fp6.ModflowGwfsto(gwf, iconvert=1, ss=Ss, sy=Sy, steady_state={0: True, 2: True}, transient={1:True})

In [ ]:
ic = fp6.ModflowGwfic(gwf, strt=top)

In [ ]:
oc = fp6.ModflowGwfoc(gwf, head_filerecord=f"{name}.hds", budget_filerecord=f"{name}.cbc", saverecord={0: [("head", "all"), ("budget", "all")]})

## Boundary conditions

### Groundwater recharge

In [ ]:
rch = fp6.ModflowGwfrcha(gwf, recharge=recharge)

### Constant head

In [ ]:
chd_spd = [(0, i, ncol-1, H_east) for i in range(nrow)]

In [ ]:
chd = fp6.ModflowGwfchd(gwf, stress_period_data=chd_spd)

### Wells

In [ ]:
well_spd = {
    0: [(0, 10, 9, -75000.)],
    2: [
        (0, 10, 9, -75000.),
        (2, 12, 4, -100000.)
    ],
}

In [ ]:
wel = fp6.ModflowGwfwel(gwf, stress_period_data=well_spd)

## Write the model files and run the simulation

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

## Get model output

In [ ]:
gwf.output.methods()

In [ ]:
gwf.output.head().get_times()

In [ ]:
gwf.output.head().get_kstpkper()

In [ ]:
kstpkper = (0,2)

In [ ]:
head = gwf.output.head().get_data(kstpkper=kstpkper)

In [ ]:
spdis = gwf.output.budget().get_data(text="SPDIS", kstpkper=kstpkper)[0]

In [ ]:
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)

## Plot model results

In [ ]:
plt_lay = 2
xsec_row = 12

### Map

In [ ]:
pmv = flopy.plot.PlotMapView(model=gwf, layer=plt_lay)
hp = pmv.plot_array(head)
pmv.plot_grid()
pmv.plot_vector(qx, qy)
cb = plt.colorbar(hp, shrink=0.75)

### Cross-section

In [ ]:
xsect = flopy.plot.PlotCrossSection(model=gwf, line={"Row": xsec_row})
csa = xsect.plot_array(head, head=head)
linecollection = xsect.plot_grid()
quiver = xsect.plot_vector(
    qx,
    qy,
    qz,
    head=head,
    hstep=2,
    normalize=True,
    color="green",
    scale=30,
    headwidth=3,
    headlength=3,
    headaxislength=3,
    zorder=10,
)

cb = plt.colorbar(csa, shrink=0.75)